# Database Systems Project
### Data parsing, processing, splitting and reorganizing and loading the data into the database


Library: Pandas

In [14]:
import pandas as pd
import numpy as np

### First, parsing of collisions2018.csv

In [15]:
dfc = pd.read_csv('CSV-2018/collisions2018.csv', dtype={'case_id': np.uint})

In [16]:
dfc.dtypes

case_id                      uint64
collision_date               object
collision_severity           object
collision_time               object
county_city_location          int64
hit_and_run                  object
jurisdiction                float64
lighting                     object
location_type                object
officer_id                   object
pcf_violation               float64
pcf_violation_category       object
pcf_violation_subsection     object
population                  float64
primary_collision_factor     object
process_date                 object
ramp_intersection           float64
road_condition_1             object
road_condition_2             object
road_surface                 object
tow_away                    float64
type_of_collision            object
weather_1                    object
weather_2                    object
dtype: object

case_id duplicates : there are 3 case_id's that appear twice, which we took the liberty to drop.

In [17]:
len(dfc['case_id'].drop_duplicates())

3678060

In [18]:
dfc = dfc.drop_duplicates('case_id')

In [19]:
len(dfc)

3678060

In [20]:
dfc['tow_away'] = dfc['tow_away'].apply(bool)

In [21]:
dfc['collision_date'] = pd.to_datetime(dfc['collision_date'])
dfc['process_date'] = pd.to_datetime(dfc['process_date'])
dfc['collision_time'] = pd.to_datetime(dfc['collision_time'])

For attributes we store their corresponding character:

In [22]:
d1 = {'property damage only': 0, 'fatal': 1, 'severe injury': 2, 'other injury': 3, 'pain': 4}
d2 = {'not hit and run': 'N', 'misdemeanor': 'M', 'felony': 'F', 'D': np.nan}
d3 = {'daylight': 'A', 'dusk or dawn': 'B', 'dark with street lights': 'C', 
      'dark with no street lights': 'D', 'dark with street lights not functioning':'E'}
d4 = {'dui': 1, 'impeding traffic': 2, 'speeding': 3, 'following too closely': 4, 'wrong side of road': 5,
      'improper passing': 6, 'unsafe lane change': 7, 'improper turning': 8, 'automobile right of way': 9,
      'pedestrian right of way': 10, 'pedestrian violation': 11, 'traffic signals and signs': 12,
      'hazardous parking': 13, 'lights': 14, 'brakes': 15, 'other equipment': 16,
      'other hazardous violation': 17, 'other than driver (or pedestrian)': 18, 'unsafe starting or backing': 19,
      'other improper driving': 22, 'pedestrian dui': 23, 'fell asleep': 24, 'unknown': 0, '21804': np.nan}
d5 = {'vehicle code violation': 'A', 'other improper driving': 'B', 'other than driver': 'C',
      'unknown': 'D', 'fell asleep': 'E'}
d6 = {'holes': 'A', 'loose material': 'B', 'obstruction': 'C', 'construction': 'D', 'reduced width': 'E',
      'flooded': 'F', 'other': 'G', 'normal': 'H'}
d7 = {'H': np.nan}
d8 = {'head-on': 'A', 'sideswipe': 'B', 'rear end': 'C', 'broadside': 'D', 'hit object': 'E', 'overturned': 'F',
      'pedestrian': 'G', 'other': 'H'}
d01 = {'highway': 'H', 'intersection': 'I', 'ramp': 'R'}
d02 = {'dry': 'A', 'wet': 'B', 'snowy': 'C', 'slippery': 'D', 'H': np.nan}
d03 = {'clear': 'A', 'cloudy': 'B', 'raining': 'C', 'snowing': 'D', 'fog': 'E', 'other': 'F', 'wind': 'E'}
d04 = {False: 0, True: 1}

In [23]:
dfc = dfc.replace({'collision_severity': d1, 'hit_and_run': d2, 'lighting': d3,
                   'pcf_violation_category': d4, 'primary_collision_factor': d5, 
                   'road_condition_1': d6, 'road_condition_2': d6, 'road_surface': d02,
                   'type_of_collision': d8, 'location_type': d01, 'weather_1': d03, 'weather_2': d03,
                   'tow_away': d04})
dfc['collision_severity'] = dfc['collision_severity'].apply(int)
dfc['pcf_violation_category'] = dfc['pcf_violation_category'].apply(float)
dfc['tow_away'] = dfc['tow_away'].apply(int)

### Now, parsing of parties2018.csv

In [32]:
dfp = pd.read_csv('CSV-2018/parties2018.csv', dtype={'case_id': np.uint, 'id': np.uint})

In [33]:
dfp.dtypes

at_fault                          int64
case_id                          uint64
cellphone_use                    object
financial_responsibility         object
hazardous_materials              object
id                               uint64
movement_preceding_collision     object
other_associate_factor_1         object
other_associate_factor_2         object
party_age                       float64
party_drug_physical              object
party_number                      int64
party_safety_equipment_1         object
party_safety_equipment_2         object
party_sex                        object
party_sobriety                   object
party_type                       object
school_bus_related               object
statewide_vehicle_type           object
vehicle_make                     object
vehicle_year                    float64
dtype: object

We fix some type issues (True/False values)

In [34]:
dfp['at_fault'] = dfp['at_fault'].apply(int)
print('done')

done


We translate hazardous_materials/school_bus_related to: NaN -> False, 'A'/'E' -> True

In [35]:
dfp = dfp.replace({'hazardous_materials': {np.nan: 0, 'A': 1}, 'school_bus_related': {np.nan: 0, 'E': 1}})
dfp['hazardous_materials'] = dfp['hazardous_materials'].apply(int)
dfp['school_bus_related'] = dfp['school_bus_related'].apply(int)
print('done')

done


Then we translate cellphone_use, financial_responsibility, other_associate_factor_1/2, party_drug_physical, party_safety_equipment_1/2, party_sobriety,  letters to strings

In [36]:
d9 = {'1': 'B', '2': 'C', '3': 'D'}
d10 = {'stopped': 'A', 'proceeding straight': 'B', 'ran off road': 'C', 'making right turn': 'D',
       'making left turn': 'E', 'making u-turn': 'F', 'backing': 'G', 'slowing/stopping': 'H',
       'passing other vehicle': 'I', 'changing lanes': 'J', 'parking maneuver': 'K', 'entering traffic': 'L',
       'other unsafe turning': 'M', 'crossed into opposing lane': 'N', 'parked': 'O', 'merging': 'P',
       'traveling wrong way': 'Q', 'other': 'R'}
d11 = {'G': np.nan}
d12 = {'passenger car': 'A', 'passenger car with trailer': 'B', 'motorcycle or scooter': 'C',
      'pickup or panel truck': 'D', 'pickup or panel truck with trailer': 'E', 'truck or truck tractor': 'F',
       'truck or truck tractor with trailer': 'G', 'schoolbus': 'H', 'other bus': 'I', 'emergency vehicle': 'J',
       'highway construction equipment': 'K', 'bicycle': 'L', 'other vehicle': 'M', 'pedestrian': 'N', 'moped': 'O'}
# tried to correct as much errors as I could catch, went through all possible values
d13 = {'DODG': 'DODGE', 'TOYTA': 'TOYOTA', 'NISS': 'NISSAN', 'HOND': 'HONDA', 'MAZD': 'MAZDA',
       'ALFA ROMERO': 'ALFA ROMEO', 'WHITE GMC': 'GMC', 'WHITE VOLVO': 'VOLVO', 'MERCEDES BENZ': 'MERCEDES-BENZ',
       'MERCEDES' : 'MERCEDES-BENZ'}
d14 = {'female': 'F', 'male': 'M'}
d15 = {'driver': 1, 'pedestrian': 2, 'parked vehicle': 3, 'bicyclist': 4, 'other': 5}

In [37]:
dfp = dfp.replace({'cellphone_use': d9, 'movement_preceding_collision': d10, 'party_drug_physical': d11,
                   'statewide_vehicle_type': d12, 'vehicle_make': d13, 'party_sex': d14, 'party_type': d15})

In [38]:
dfp.dtypes

at_fault                          int64
case_id                          uint64
cellphone_use                    object
financial_responsibility         object
hazardous_materials               int64
id                               uint64
movement_preceding_collision     object
other_associate_factor_1         object
other_associate_factor_2         object
party_age                       float64
party_drug_physical              object
party_number                      int64
party_safety_equipment_1         object
party_safety_equipment_2         object
party_sex                        object
party_sobriety                   object
party_type                      float64
school_bus_related                int64
statewide_vehicle_type           object
vehicle_make                     object
vehicle_year                    float64
dtype: object

### Finally, parsing of victims.csv

In [39]:
dfv = pd.read_csv('CSV-2018/victims2018.csv', dtype={'case_id': np.uint, 'id': np.uint})

/Users/jean-baptistemichel/opt/anaconda3/envs/db-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [40]:
dfv.dtypes

case_id                       uint64
id                            uint64
party_number                   int64
victim_age                   float64
victim_degree_of_injury       object
victim_ejected               float64
victim_role                    int64
victim_safety_equipment_1     object
victim_safety_equipment_2     object
victim_seating_position      float64
victim_sex                    object
dtype: object

In [41]:
d14 = {'no injury': 0, 'killed': 1, 'severe injury': 2, 'other visible injury': 3, 'complaint of pain': 4,
       'suspected serious injury': 5, 'suspected minor injury': 6, 'possible injury': 7}
d15 = {4.0: np.nan}
d16 = {'female': 'F', 'male': 'M'}
d17 = {1.0: '1', 2.0: '2', 3.0: '3', 4.0:'4', 5.0: '5', 6.0: '6', 7.0: '7', 8.0:'8', 9.0: '9', 0.0: '0', np.nan:''}

In [42]:
dfv = dfv.replace({'victim_degree_of_injury': d14, 'victim_ejected': d15, 'victim_sex': d16,
                   'victim_seating_position': d17})
dfv['victim_degree_of_injury'] = dfv['victim_degree_of_injury'].apply(int)
dfv['victim_role'] = dfv['victim_role'].apply(int)
dfv['victim_seating_position'] = dfv['victim_seating_position'].apply(str)

## Preparing the data tables

First, collisions:

In [14]:
collisions = dfc.drop(['road_condition_1', 'road_condition_2', 'weather_1', 'weather_2',
                       'county_city_location', 'location_type', 'population', 'pcf_violation',
                       'pcf_violation_category', 'pcf_violation_subsection', 'primary_collision_factor'], axis=1)

Then, we create the road_condition, PCF and weather entities:

In [12]:
rc1 = dfc[['case_id','road_condition_1']].rename(columns={"road_condition_1": "road_condition"})
rc2 = dfc[['case_id','road_condition_2']].rename(columns={"road_condition_2": "road_condition"})

road_condition_collision = rc1.append(rc2).dropna().drop_duplicates()

In [13]:
# useless imo
road_conditions = dfc[['road_condition_1']].rename(columns={"road_condition_1": "road_condition"}).drop_duplicates().dropna()

In [24]:
pcf = dfc[['case_id', 'pcf_violation', 'pcf_violation_category',
           'pcf_violation_subsection', 'primary_collision_factor']]

In [25]:
len(pcf)

3678060

In [14]:
w1 = dfc[['case_id','weather_1']].rename(columns={"weather_1": "weather"})
w2 = dfc[['case_id','weather_2']].rename(columns={"weather_2": "weather"})

with_weather = w1.append(w2).dropna().drop_duplicates()

In [15]:
# useless imo
weather = with_weather[['weather']].drop_duplicates()

Finally, the location entity + relationship table

In [16]:
at_location = dfc[['case_id', 'county_city_location', 'location_type', 'population']]

In [17]:
# useless imo
location = dfc[['county_city_location', 'location_type', 'population']].dropna().drop_duplicates()

In [22]:
road_condition_collision.to_csv('road_cond.csv', index=False)

In [23]:
with_weather.to_csv('col_weather.csv', index=False)

In [24]:
at_location.to_csv('at_location.csv', index=False)

In [ ]:
pcf.to_csv('pcf.csv', index=False)

Now, parties:

In [43]:
parties = dfp.drop(['other_associate_factor_1', 'other_associate_factor_2', 'party_safety_equipment_1',
                    'party_safety_equipment_2', 'statewide_vehicle_type', 'vehicle_make', 'vehicle_year'], axis=1)
parties = parties.rename(columns={"id": "pid"})

Then, we create the other_associated_factor, party_safety_equipment and party_vehicle entities

In [44]:
oaf1 = dfp[['id', 'other_associate_factor_1']].rename(columns={"other_associate_factor_1": "other_associated_factor"})
oaf2 = dfp[['id', 'other_associate_factor_2']].rename(columns={"other_associate_factor_2": "other_associated_factor"})

other_associated_factor = oaf1.append(oaf2).dropna().drop_duplicates().rename(columns={"id": "pid"})

In [45]:
pse1 = dfp[['id', 'party_safety_equipment_1']].rename(columns={"party_safety_equipment_1": "party_safety_equipment"})
pse2 = dfp[['id', 'party_safety_equipment_2']].rename(columns={"party_safety_equipment_2": "party_safety_equipment"})

party_safety_equipment = pse1.append(pse2).dropna().drop_duplicates().rename(columns={"id": "pid"})

In [46]:
# useless imo
safety_equipment = party_safety_equipment[['party_safety_equipment']].drop_duplicates()

In [47]:
party_vehicle = dfp[['id', 'statewide_vehicle_type', 'vehicle_make', 'vehicle_year']].rename(columns={'id':'pid'})

In [ ]:
## Do we also create a useless vehicle table ?

In [ ]:
parties.to_csv('party.csv', index=False)

In [ ]:
other_associated_factor.to_csv('oaf.csv', index=False)

In [ ]:
party_safety_equipment.to_csv('pse.csv', index=False)

In [ ]:
party_vehicle.to_csv('vehicle.csv', index=False)

Finally, victims:

In [48]:
victims = dfv.drop(['victim_safety_equipment_1', 'victim_safety_equipment_2'],axis=1).rename(columns={"id":"vid"})

Then, we create the victim_safety_equipment entity:

In [49]:
vse1 = dfv[['id', 'victim_safety_equipment_1']].rename(columns={"victim_safety_equipment_1": "victim_safety_equipment"})
vse2 = dfv[['id', 'victim_safety_equipment_2']].rename(columns={"victim_safety_equipment_2": "victim_safety_equipment"})

victim_safety_equipment = vse1.append(vse2).dropna().drop_duplicates().rename(columns={"id":"vid"})

In [25]:
victims.to_csv('victim.csv', index=False)

In [26]:
victim_safety_equipment.to_csv('vse.csv', index=False)